# LRR Annotator

Below is example usage of our LRR annotator. 


In [24]:
from loader import Loader
from analyzer import Analyzer
from plotter import Plotter

In [31]:
L = Loader()
L.load_batch('./pdb')

A = Analyzer()
P = Plotter()

AttributeError: module 'posixpath' has no attribute 'readlines'

In [75]:
from common import *

class Loader:
	def __init__(self):
		self.structures = {}


	def load_batch(self, directory):
		"""Loads batch of PDB files from specified directory and stores
		them in the self.structures dictionary, where they can be looked up
		by filename.

		Args:
			directory (str): Path to folder containing .pdb files.
		"""
		parser = PDBParser()
		for filename in os.listdir(directory):
			if filename.endswith('.pdb'):
				path = os.path.join(directory, filename)
				key = os.path.splitext(filename)[0]
				chain = next(parser.get_structure(key, path).get_chains())
				self.structures[key] = np.array([np.array(list(residue["CA"].get_vector())) for residue in chain.get_residues()])	


	def load_single(self, directory, filename):
		"""Loads single PDB file from specified path, stores it in the
		self.structures dictionary

		Args:
			pdb_file (str): Path to .pdb file.
		"""
		parser = PDBParser()
		assert filename in os.listdir(directory)

		if filename.endswith('.pdb'):
			path = os.path.join(directory, filename)
			key = os.path.splitext(pdb_file)[0]
			chain = next(parser.get_structure(key, path).get_chains())
			self.structures[key] = np.array([np.array(list(residue["CA"].get_vector())) for residue in chain.get_residues()])	


In [74]:
a = './pdb/AT5G47280.1.pdb'
a.ends_with('.pdb')

AttributeError: 'str' object has no attribute 'ends_with'

In [76]:
L = Loader()
# L.load_batch('./pdb/')

In [77]:
L.load_single('./pdb/AT5G47280.1.pdb')
# plt.plot(L.structures['AT5G47280.1'])

In [78]:
L.structures

{'./pdb/AT5G47280.1': array([[-18.75600052, -22.66900063,  -2.69099998],
        [-19.69799995, -19.45199966,  -4.60200024],
        [-23.38599968, -20.55900002,  -4.90500021],
        ...,
        [  7.49700022,  -8.14999962,  10.97000027],
        [ 10.54599953, -10.30500031,  10.8760004 ],
        [ 12.35299969,  -8.45400047,   7.97599983]])}

In [71]:
os.listdir('./pdb/AT5G47280.1.pdb')

['AT5G47280.1.pdb',
 'AT1G15890.1.pdb',
 'AT3G04220.1.pdb',
 'AT5G46470.1.pdb',
 'AT5G38350.1.pdb',
 'AT5G41750.1.pdb',
 'AT5G66910.1.pdb',
 'AT1G56520.1.pdb',
 'AT1G31540.2.pdb',
 'AT3G46530.1.pdb',
 'AT5G41750.2.pdb',
 'AT5G11250.1.pdb',
 'AT3G14470.1.pdb',
 'AT1G56520.2.pdb',
 'AT4G12010.1.pdb',
 'AT1G31540.1.pdb',
 'AT4G16960.1.pdb',
 'AT4G36150.1.pdb',
 'AT1G61180.2.pdb',
 'AT5G18360.1.pdb',
 'AT1G12290.1.pdb',
 'AT1G10920.1.pdb',
 'AT5G46490.2.pdb',
 'AT1G63740.1.pdb',
 'AT4G16940.1.pdb',
 'AT1G59218.2.pdb',
 'AT5G51630.1.pdb',
 'AT5G45250.1.pdb',
 'AT5G17680.1.pdb',
 'AT5G43740.1.pdb',
 'AT3G44400.2.pdb',
 'AT4G16860.1.pdb',
 'AT1G61180.1.pdb',
 'AT5G47260.1.pdb',
 'AT5G46260.1.pdb',
 'AT1G12290.2.pdb',
 'AT5G46450.1.pdb',
 'AT5G51630.3.pdb',
 'AT1G10920.2.pdb',
 'AT5G40060.1.pdb',
 'AT1G59218.1.pdb',
 'AT5G51630.2.pdb',
 'AT5G43740.2.pdb',
 'AT5G17970.1.pdb',
 'AT1G59124.1.pdb',
 'AT1G61300.1.pdb',
 'AT1G63870.1.pdb',
 'AT3G44400.1.pdb',
 'AT5G17890.1.pdb',
 'AT5G45210.1.pdb',
